In [1]:
%matplotlib inline
import numpy as np
import sklearn.decomposition
import sklearn.feature_extraction
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import bottleneck as bn
import itertools
from sklearn.cross_validation import KFold

###Fake Testing

In [2]:
# arbitrary 6x3 matrix (constructed from an example at http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html with a random column concatenated)
M = np.concatenate((np.array([[1,1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]]),map(lambda x : [x], np.round(np.random.random_sample(6),1)+1)),1)
M
H = sklearn.decomposition.NMF(n_components=2).fit(M).components_
W = sklearn.decomposition.NMF(n_components=2).fit_transform(M)
print H
print
print W

[[  3.10114034e+00   4.83338531e-01   6.39050455e-01]
 [  1.74830868e-06   8.29630628e-01   1.46613606e+00]]

[[ 0.32461333  0.81094869]
 [ 0.64345156  0.97058872]
 [ 0.96691658  0.92818676]
 [ 1.29143483  0.30334212]
 [ 1.61052846  0.19772203]
 [ 1.93555763 -0.        ]]


In [3]:
# a 20x4 example in which there are explicit types (that are unknown to the algorithm)
n = 20
types = np.trunc(np.random.random_sample(n)*3)
means = [[1,2,3,4],[1,5,4,1],[10,1,2,8]]
stddevs = 0.5 * np.ones([3,4])
def gen(t):
    mean = means[t]
    sd = stddevs[t]
    sz = len(mean)
    return list(np.random.randn(sz) * sd + mean)
# generate a data matrix
M = np.round(map(gen, map(int, types)),3)

M_tfidf = sklearn.feature_extraction.text.TfidfTransformer().fit_transform(M).toarray()
print M
print
print M_tfidf
print
print

# run NMF
fit = sklearn.decomposition.NMF(n_components=3).fit(M)
H = fit.components_
W = fit.transform(M)
#print H
results = list(np.concatenate((10*W,map(lambda x:[x],types)), 1))
results.sort(key = lambda row : row[3])
results = np.array(map(lambda row : list(np.round(row,3)), results))
print results
#print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), results)
# predicted clusters
print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), W)
# true clusters
print map(int, types)
# use fit from above to classify a new point
print fit.transform([[1,2,4,5]])

print
print

fit = sklearn.decomposition.NMF(n_components=3).fit(M_tfidf)
H = fit.components_
W = fit.transform(M_tfidf)
#print H
results = list(np.concatenate((10*W,map(lambda x:[x],types)), 1))
results.sort(key = lambda row : row[3])
results = np.array(map(lambda row : list(np.round(row,3)), results))
print results
#print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), results)
# predicted clusters
print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), W)
# true clusters
print map(int, types)

[[  0.562   5.141   4.129   1.558]
 [  1.6     1.386   2.468   4.057]
 [  9.212   0.866   2.135   8.239]
 [  0.311   1.533   1.661   3.613]
 [  1.686   1.816   3.611   4.524]
 [  0.894   2.137   3.459   3.962]
 [  0.876   2.042   3.435   3.907]
 [  1.416   1.555   3.784   3.591]
 [  0.751   5.301   3.53    1.438]
 [  0.905   5.607   4.625   1.887]
 [ 10.037   1.016   1.883   8.234]
 [  1.515   5.462   3.475   0.451]
 [  9.206   1.244   2.145   8.246]
 [  1.541   4.772   4.04    1.036]
 [  1.31    2.317   3.579   3.279]
 [ 10.281   0.732   1.87    8.117]
 [  1.077   5.198   4.031   0.158]
 [ 10.159   0.503   1.791   8.835]
 [  0.943   1.846   3.358   4.413]
 [  0.829   5.285   3.624   1.092]]

[[ 0.08266339  0.75617881  0.60732588  0.22916292]
 [ 0.307742    0.2665815   0.47469203  0.7803183 ]
 [ 0.73275087  0.06888431  0.16982448  0.65535545]
 [ 0.07278065  0.35875479  0.38870953  0.84551929]
 [ 0.26776633  0.28841261  0.57349005  0.71849044]
 [ 0.15555914  0.3718455   0.60187814  0.68

C:\Users\Madhu\Anaconda\lib\site-packages\sklearn\decomposition\nmf.py:532: UserWarning: Iteration limit reached during fit. Solving for W exactly.
  warnings.warn("Iteration limit reached during fit. Solving for W exactly.")


###Fitting Model with Training Data

In [4]:
%%time
# read in training data from csv
noun_train_mat = np.loadtxt("noun_train_mat.csv", delimiter = ",")
# use TF-IDF to scale each document's vector to have norm 1 and place a lower weight on very common words
tf_idf_fit = sklearn.feature_extraction.text.TfidfTransformer().fit(noun_train_mat)
noun_train_mat = tf_idf_fit.transform(noun_train_mat).toarray()

Wall time: 30.4 s


In [5]:
%%time
# compute NMF fit
NMF_fit = sklearn.decomposition.NMF(n_components=14, init='nndsvda').fit(noun_train_mat)
H = NMF_fit.components_
W = NMF_fit.transform(noun_train_mat)
# contains a tuple (i,j) if document i is in cluster j, for each document
clusters = map(np.argmax, W)
# list of the documents in each cluster
cluster_lists = [[i for i,j in enumerate(clusters) if j==cluster] for cluster in range(14)]

Wall time: 1min 6s


In [6]:
# classify each document into the category that fits it best
print 'Number of documents per category:', [sum([x==i for x in clusters]) for i in range(14)]

Number of documents per category: [100, 185, 74, 59, 68, 145, 44, 68, 88, 53, 35, 13, 26, 152]


###Key Words for Each Cluster

In [7]:
# load vocab from csv
noun_vocab = np.loadtxt("noun_vocab.csv", delimiter=",", dtype="str")
noun_vocab = [(int(i),j) for i,j in noun_vocab]
id2noun = dict(noun_vocab)

In [8]:
# find and output the 5 most important words for each category
num_best = 5
best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
best_words = [[id2noun[i] for i in lst] for lst in best_indices]
best_words

[['court', 'district', 'petitioner', 'appeal', 'habea'],
 ['act', 'respondent', 'action', 'court', 'title'],
 ['labor', 'union', 'board', 'employee', 'employer'],
 ['warrant', 'police', 'search', 'officer', 'petitioner'],
 ['property', 'tax', 'revenue', 'income', 'busines'],
 ['jury', 'sentence', 'defendant', 'trial', 'offense'],
 ['carrier', 'railroad', 'icc', 'rate', 'commerce'],
 ['attorney', 'alien', 'general', 'brief', 'cause'],
 ['commission', 'price', 'act', 'sale', 'company'],
 ['contract', 'arbitration', 'agreement', 'government', 'contractor'],
 ['plan', 'board', 'school', 'student', 'education'],
 ['patent', 'invention', 'art', 'claim', 'royalty'],
 ['master', 'decree', 'orig ', 'entry', 'boundary'],
 ['state', 'court', 'statute', 'law', 'amendment']]

In [9]:
# can probably ignore this stuff -- it's a different way to figure out the best words in each category, but it's not working
# and produces nonsense
width = len(M[0])
best_words = np.zeros(14)
for i in range(14):
    # arbitrary high initial value
    w = np.ones(width) * 5000
    #w_val = np.ones(width)
    for j in range(width):
        for k in range(14):
            if i <> k and H[k][j] > 0:
                if w[j] >= float(H[i][j])/H[k][j]:
                    w[j] = float(H[i][j])/H[k][j]
    best_words[i] = np.argmax(w)
print map(lambda x : id2noun[int(x)], best_words)

['narcotic', 'fdca', 'middleman', 'narcotic', 'city the', 'middleman', 'city the', 'middleman', 'fdca', 'narcotic', 'city the', 'fdca', 'city the', 'city the']


###Comparing with Supreme Court Database's Topic Areas

In [10]:
# read in SC Database's issue areas from csv
noun_train_issue_areas = np.loadtxt("noun_train_issue_areas.csv", delimiter = ",", dtype="int")
# zero-index the array
noun_train_issue_areas = noun_train_issue_areas - 1
print 'Number of documents per category:', [sum([x==i for x in noun_train_issue_areas]) for i in range(14)]

# convert the nx1 issue areas vector into a nx14 matrix of dummies, in case that's ever useful
noun_train_issue_areas_dummy = np.array(map(lambda area : np.eye(1,14,area)[0], noun_train_issue_areas))

Number of documents per category: [238, 188, 96, 36, 10, 8, 64, 242, 131, 56, 10, 30, 1, 0]


####Assignment method 1: use the cluster assignments computed above to find the best SCDB category match

In [14]:
# create a 14x14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
# related-ness between each cluster pair
compare_mat = map(lambda r : map(int, r), np.zeros((14,14)))
# first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
for i,j in zip(clusters, noun_train_issue_areas):
    compare_mat[i][j] += 1
# normalize each row to have a sum of 1
compare_mat = map(lambda row : map(float,row) / sum(row), np.array(compare_mat))

# assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
assignments = map(np.argmax, compare_mat)
print assignments
# this would be the equivalent way of assigning each SCDB cluster to an NMF cluster; this is printed here but not used anywhere
print map(np.argmax, np.array(compare_mat).T)

[8, 7, 6, 0, 7, 0, 7, 1, 7, 7, 1, 7, 10, 2]
[3, 10, 13, 13, 13, 7, 2, 11, 0, 12, 12, 4, 4, 0]


####Assignment method 2: use the rows of the W matrix (from NMF) to find the best SCDB category match for each cluster

In [12]:
# Difference compared to above: Instead of just thinking of each case as getting assigned to a single cluster (as we did above),
# we can think of each case as having a coefficient for each cluster (from the W matrix), and increment entries of compare_mat
# using those coefficients instead of just incrementing by 1.

# create a 14x14 matrix (where each row is an SCDB cluster and each column is an NMF cluster) measuring the degree of
# related-ness between each cluster pai
compare_mat = np.zeros((14,14))
for i,j in zip(W, noun_train_issue_areas):
    # increment the jth row of compare_mat by the entire row of W corresponding to the current case
    compare_mat[j] = compare_mat[j] + i
# transpose that matrix (so that it's in the same format as the matrix in assignment method 1 above)
compare_mat = compare_mat.T

# assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
assignments = map(np.argmax, compare_mat)
print assignments

[0, 7, 6, 0, 7, 0, 7, 0, 7, 7, 1, 7, 10, 1]


####Assignment method 3: use Hungarian algorithm (which computes assignments that minimize cost) to assign each NMF cluster to an SCDB cluster (without any collisions)

In [13]:
### NOTE: probably don't run this -- it generates assignments that have very low accuracy
import munkres
m = munkres.Munkres()
assignments = dict(m.compute(-np.array(compare_mat)))
print assignments

{0: 8, 1: 1, 2: 6, 3: 4, 4: 11, 5: 0, 6: 3, 7: 5, 8: 7, 9: 9, 10: 13, 11: 12, 12: 10, 13: 2}


####Assign an SCDB category to each case, using an assignment array created above 

In [15]:
# use the assignments above to convert the clusters into equivalent SCDB categories for each document
new_clusters = map(lambda cluster : assignments[cluster], clusters)
# compute and output accuracy rate
correct = map(lambda (c1,c2) : c1==c2, zip(new_clusters, noun_train_issue_areas))
print float(sum(correct)) / len(correct)

0.490990990991


###Use Rand Index to Compute Accuracy on Training Data

Each pair of documents is called a true positive ($tp$) if the two documents end up in the same cluster correctly, a true negative ($tn$) if the documents end up in different clusters correctly, a false positive ($fp$) if the documents are placed in the same cluster but should have been in different clusters, and a false negative ($fn$) if the documents are placed in different clusters but should have been in the same cluster.

The Rand index (measuring the similarity between the NMF clustering and the SCDB categories) is defined as $\frac{tp+tn}{tp+tn+fp+fn}$.

In [16]:
%%time
if 'cluster_pairs_model' in globals():
    del cluster_pairs_model
if 'cluster_pairs_actual' in globals():
    del cluster_pairs_actual

# use itertools to create pairs of documents, using the NMF clusters and SCDB categories
cluster_pairs_model = itertools.product(clusters, clusters)
cluster_pairs_actual = itertools.product(noun_train_issue_areas, noun_train_issue_areas)
# loop through all document pairs to compute tp, tn, fp, and fn
(tp,tn,fp,fn) = 0, 0, 0, 0
for pair1,pair2 in zip(cluster_pairs_model, cluster_pairs_actual):
    if (pair1[0]==pair1[1]):
        if (pair2[0]==pair2[1]):
            tp += 1
        else:
            fp += 1
    else:
        if (pair2[0]==pair2[1]):
            fn += 1
        else:
            tn += 1

# output tp, tn, fp, and fn
print tp,tn,fp,fn
# compute and output Rand index using formula from above
rand_index_train = float(tp+tn)/(tp+tn+fp+fn)
print rand_index_train

39504 963864 81614 147118
0.814355977599
Wall time: 1.77 s


###Applying Model to Test Data

In [17]:
%%time
# read in test data from csv
noun_test_mat = np.loadtxt("noun_test_mat.csv", delimiter = ",")
# use TF-IDF fit from training data to scale each document's vector to have norm 1 and place a lower weight on very common words
noun_test_mat = tf_idf_fit.transform(noun_test_mat).toarray()

Wall time: 14.6 s


In [18]:
%%time
# use NMF fit from training data to cluster test observations
W_test = NMF_fit.transform(noun_test_mat)
clusters_test = map(np.argmax, W_test)
cluster_lists_test = [[i for i,j in enumerate(clusters_test) if j==cluster] for cluster in range(14)]

Wall time: 3.17 s


###Compare Test Data Results to SC Database Categories

In [19]:
# read in SC Database's issue areas from csv
noun_test_issue_areas = np.loadtxt("noun_test_issue_areas.csv", delimiter = ",", dtype="int")
# zero-index the array
noun_test_issue_areas = noun_test_issue_areas - 1
print 'Number of documents per category:', [sum([x==i for x in noun_test_issue_areas]) for i in range(14)]

Number of documents per category: [104, 95, 26, 22, 3, 7, 18, 96, 63, 25, 5, 17, 4, 0]


In [20]:
# use the assignments above to convert the clusters into equivalent SCDB categories for each document
new_clusters_test = map(lambda cluster : assignments[cluster], clusters_test)
# compute and output accuracy rate
correct_test = map(lambda (c1,c2) : c1==c2, zip(new_clusters_test, noun_test_issue_areas))
print float(sum(correct_test)) / len(correct_test)

0.455670103093


###Use Rand Index to Compute Accuracy on Test Data

In [21]:
%%time
if 'cluster_test_pairs_model' in globals():
    del cluster_test_pairs_model
if 'cluster_test_pairs_actual' in globals():
    del cluster_test_pairs_actual

# use itertools to create pairs of documents, using the NMF clusters and SCDB categories
cluster_test_pairs_model = itertools.product(clusters_test, clusters_test)
cluster_test_pairs_actual = itertools.product(noun_test_issue_areas, noun_test_issue_areas)
# loop through all document pairs to compute tp, tn, fp, and fn
(tp,tn,fp,fn) = 0, 0, 0, 0
for pair1,pair2 in zip(cluster_test_pairs_model, cluster_test_pairs_actual):
    if (pair1[0]==pair1[1]):
        if (pair2[0]==pair2[1]):
            tp += 1
        else:
            fp += 1
    else:
        if (pair2[0]==pair2[1]):
            fn += 1
        else:
            tn += 1

# output tp, tn, fp, and fn
print tp,tn,fp,fn
# compute and output Rand index using formula from above
rand_index_test = float(tp+tn)/(tp+tn+fp+fn)
print rand_index_test

7869 184414 15288 27654
0.817442873844
Wall time: 356 ms


###Compute Optimal Number of Clusters on Training Data (based on SCDB)
Note: In reality, we should do this using one or more validation sets; I've just done this for the time being.

In [83]:
%%time
num_clusters_lst = [5, 10, 14, 15, 20, 25, 30]

results = dict()
for num_clusters in num_clusters_lst:
    NMF_fit = sklearn.decomposition.NMF(n_components=num_clusters, init='nndsvda').fit(noun_train_mat)
    H = NMF_fit.components_
    W = NMF_fit.transform(noun_train_mat)
    # contains a tuple (i,j) if document i is in cluster j, for each document
    clusters = map(np.argmax, W)
    # list of the documents in each cluster
    cluster_lists = [[i for i,j in enumerate(clusters) if j==cluster] for cluster in range(num_clusters)]
    
    # find the 10 most important words for each category
    num_best = 10
    best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
    best_words = [[id2noun[i] for i in lst] for lst in best_indices]
    
    # create a k x 14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
    # related-ness between each cluster pair
    compare_mat = map(lambda r : map(int, r), np.zeros((num_clusters,14)))
    # first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
    for i,j in zip(clusters, noun_train_issue_areas):
        compare_mat[i][j] += 1
    # normalize each row to have a sum of 1
    compare_mat = map(lambda row : map(float,row) / sum(row), np.array(compare_mat))
    # assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
    assignments = map(np.argmax, compare_mat)
    # use the assignments above to convert the clusters into equivalent SCDB categories for each document
    new_clusters = map(lambda cluster : assignments[cluster], clusters)
    # compute and output accuracy rate
    correct = map(lambda (c1,c2) : c1==c2, zip(new_clusters, noun_train_issue_areas))
    accuracy = float(sum(correct)) / len(correct)

    if 'cluster_pairs_model' in globals():
        del cluster_pairs_model
    if 'cluster_pairs_actual' in globals():
        del cluster_pairs_actual
    # use itertools to create pairs of documents, using the NMF clusters and SCDB categories
    cluster_pairs_model = itertools.product(clusters, clusters)
    cluster_pairs_actual = itertools.product(noun_train_issue_areas, noun_train_issue_areas)
    # loop through all document pairs to compute tp, tn, fp, and fn
    (tp,tn,fp,fn) = 0, 0, 0, 0
    for pair1,pair2 in zip(cluster_pairs_model, cluster_pairs_actual):
        if (pair1[0]==pair1[1]):
            if (pair2[0]==pair2[1]):
                tp += 1
            else:
                fp += 1
        else:
            if (pair2[0]==pair2[1]):
                fn += 1
            else:
                tn += 1
    # compute Rand index using formula from above
    rand_index = float(tp+tn)/(tp+tn+fp+fn)
    
    results[num_clusters] = (NMF_fit, clusters, cluster_lists, best_words, new_clusters, accuracy, (tp,tn,fp,fn), rand_index)


Wall time: 21min 16s


In [89]:
[results[num_clusters][5] for num_clusters in num_cluster_lst]

[0.4315315315315315,
 0.4828828828828829,
 0.49099099099099097,
 0.5198198198198198,
 0.5423423423423424,
 0.5423423423423424,
 0.5387387387387388]

##Precedents

In [22]:
%%time
# read in training data from csv
prec_train_mat = np.loadtxt("prec_train_mat.csv", delimiter = ",")
# use TF-IDF to scale each document's vector to have norm 1 and place a lower weight on very common words
tf_idf_fit = sklearn.feature_extraction.text.TfidfTransformer().fit(prec_train_mat)
prec_train_mat = tf_idf_fit.transform(prec_train_mat).toarray()

Wall time: 6.96 s


In [23]:
%%time
# compute NMF fit
NMF_fit = sklearn.decomposition.NMF(n_components=14, init='nndsvda').fit(prec_train_mat)
H = NMF_fit.components_
W = NMF_fit.transform(prec_train_mat)
# contains a tuple (i,j) if document i is in cluster j, for each document
clusters = map(np.argmax, W)
# list of the documents in each cluster
cluster_lists = [[i for i,j in enumerate(clusters) if j==cluster] for cluster in range(14)]

Wall time: 3.76 s


In [24]:
# classify each document into the category that fits it best
print 'Number of documents per category:', [sum([x==i for x in clusters]) for i in range(14)]

Number of documents per category: [201, 89, 6, 5, 8, 24, 17, 18, 115, 73, 114, 118, 126, 172]


In [25]:
# load vocab from csv
prec_vocab = np.genfromtxt("prec_vocab.csv", delimiter=",", dtype="str")
#prec_vocab = dict(list(enumerate(prec_vocab)))
prec_vocab = [(int(i),j) for i,j in prec_vocab]
id2prec = dict(prec_vocab)

In [26]:
# find and output the 5 most important words for each category
num_best = 5
best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
best_words = [[id2prec[i] for i in lst] for lst in best_indices]
best_words

[['co. v. national',
  'states v. interstate',
  'co. v. united',
  'states v. el',
  'corp. v. el'],
 ['states v. detroit',
  'whitfield v. united',
  'statessyllabuskloeckner v. solis;',
  'association v. harris;',
  'commission v. abercrombie'],
 ['in v. united',
  'limitations.mcmahon v. united',
  'corporation v. united',
  'affirmed.pinkerton v. united',
  'co. v. united'],
 ['states v. california;',
  'trop v. dulles',
  'states v. louisiana',
  'lessee v. hagan;',
  'coleman v. jiffy'],
 ['younger v. harris;',
  'pennzoil v. texaco;',
  'samuels v. mackell;',
  'steffel v. thompson;',
  'byrne v. karalexis'],
 ['brown v. board',
  'green v. county',
  'bolling v. sharpe',
  'grutter v. bollinger;',
  'in v. fcc;'],
 ['al. v. united',
  'states v. mitchell;',
  'bragdon v. abbott;',
  'states v. grinnell',
  'engel v. davenport;'],
 ['carroll v. united',
  'florida v. whitecertiorari',
  'corp. v. united',
  'statute.fogarty v. united',
  'corp. v. cbs;'],
 ['bivens v. six',
  '

In [27]:
# read in SC Database's issue areas from csv
prec_train_issue_areas = np.loadtxt("prec_train_issue_areas.csv", delimiter = ",", dtype="int")
# zero-index the array
prec_train_issue_areas = prec_train_issue_areas - 1
print 'Number of documents per category:', [sum([x==i for x in prec_train_issue_areas]) for i in range(14)]

# convert the nx1 issue areas vector into a nx14 matrix of dummies, in case that's ever useful
prec_train_issue_areas_dummy = np.array(map(lambda area : np.eye(1,14,area)[0], prec_train_issue_areas))

Number of documents per category: [234, 195, 75, 36, 10, 13, 56, 235, 131, 52, 11, 33, 5, 0]


In [28]:
# create a 14x14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
# related-ness between each cluster pair
compare_mat = map(lambda r : map(int, r), np.zeros((14,14)))
# first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
for i,j in zip(clusters, prec_train_issue_areas):
    compare_mat[i][j] += 1
# normalize each row to have a sum of 1
compare_mat = map(lambda row : map(float,row) / sum(row), np.array(compare_mat))

### commented out because I realized this method of assignment does the same thing as the one-line assignment below
#assignments = -1 * np.ones(14)
#compare_mat_flat = [x for lst in compare_mat for x in lst]
#while min(assignments) == -1:
#    max_ind = np.argmax(compare_mat_flat)
#    row = max_ind / 14
#    if assignments[row] == -1:
#        column = max_ind - 14 * row
#        assignments[row] = column
#    else:
#        compare_mat_flat[max_ind] = -1
#assignments = map(int, assignments)
#print assignments

# assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
assignments = map(np.argmax, compare_mat)
print assignments
# this would be the equivalent way of assigning each SCDB cluster to an NMF cluster; this is printed here but not used anywhere
print map(np.argmax, np.array(compare_mat).T)

[7, 0, 7, 9, 8, 2, 1, 0, 7, 7, 0, 0, 7, 0]
[1, 6, 5, 6, 6, 13, 12, 2, 4, 3, 10, 2, 6, 0]


In [29]:
# use the assignments above to convert the clusters into equivalent SCDB categories for each document
new_clusters = map(lambda cluster : assignments[cluster], clusters)
# compute and output accuracy rate
correct = map(lambda (c1,c2) : c1==c2, zip(new_clusters, prec_train_issue_areas))
print float(sum(correct)) / len(correct)

0.271639042357


###Cross-Validation

In [30]:
def cv_results_unsupervised(predict, x, y, score_func, nfolds=5):
    results = []
    for train, test in KFold(len(y), nfolds):
        pred = predict(x[train],y[train],x[test])#clf.fit(x[train]).transform(x[test])
        results += [score_func(pred, y[test])] # evaluate score function on held-out data
    return results

In [43]:
# Fits an NMF model with n_components clusters on the data in x_train and y_train, and use that to predict clusters for x_test.
# (Since NMF is unsupervised, the data in y_train is only used to create a mapping from NMF clusters to SCDB clusters, not to
# actually fit the model.)
def fit_transform(x_train, y_train, x_test, n_components):
    # compute an NMF fit on x_train, and produce
    fit = sklearn.decomposition.NMF(n_components=n_components, init='nndsvda').fit(x_train)
    W_train = fit.transform(x_train)
    clusters_train = map(np.argmax, W_train)
    
    # create a 14x14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
    # related-ness between each cluster pair
    compare_mat = map(lambda r : map(int, r), np.zeros((n_components,14)))
    # first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
    for i,j in zip(clusters_train, y_train):
        compare_mat[i][j] += 1
    assignments = map(np.argmax, compare_mat)
    
    W_test = fit.transform(x_test)
    clusters_test = map(np.argmax, W_test)
    # use the assignments above to convert the clusters into equivalent SCDB categories for each document
    new_clusters = map(lambda cluster : assignments[cluster], clusters_test)
    return new_clusters

In [32]:
def accuracy(predicted,actual):
    correct = map(lambda (c1,c2) : c1==c2, zip(predicted,actual))
    #print type(correct),correct[0]
    return float(sum(correct)) / len(correct)

In [38]:
def rand_index(predicted,actual):
    # use itertools to create pairs of documents, using the NMF clusters and SCDB categories
    cluster_pairs_pred = itertools.product(predicted,predicted)
    cluster_pairs_actual = itertools.product(actual,actual)
    # loop through all document pairs to compute tp, tn, fp, and fn
    (tp,tn,fp,fn) = 0, 0, 0, 0
    for pair1,pair2 in zip(cluster_pairs_pred, cluster_pairs_actual):
        if (pair1[0]==pair1[1]):
            if (pair2[0]==pair2[1]):
                tp += 1
            else:
                fp += 1
        else:
            if (pair2[0]==pair2[1]):
                fn += 1
            else:
                tn += 1
    
    # compute and output Rand index using formula from above
    rand_index_train = float(tp+tn)/(tp+tn+fp+fn)
    return rand_index_train

In [191]:
%%time
cv_example = cv_results_unsupervised(lambda x,y,z : fit_transform(x,y,z,14), noun_train_mat, noun_train_issue_areas,
                                     lambda a,b : (accuracy(a,b), rand_index(a,b)), nfolds=5)
#cv_example = cv_results_unsupervised(sklearn.decomposition.NMF(n_components=14, init='nndsvda'), noun_train_mat,
#                                     noun_train_issue_areas, score_func, nfolds=2)

Wall time: 3min 18s


In [219]:
cv_example

[0.481981981981982,
 0.5,
 0.46396396396396394,
 0.4774774774774775,
 0.45495495495495497]

In [44]:
%%time
num_clusters_lst = [5, 10, 15, 20, 25, 30]

results = dict()
for k in num_clusters_lst:
    print 'Testing with %d clusters' % k
    results[k] = cv_results_unsupervised(lambda x,y,z : fit_transform(x,y,z,k), noun_train_mat, noun_train_issue_areas,
                                         lambda pred,act : (accuracy(pred,act), rand_index(pred,act)), nfolds=5)

Testing with 5 clusters
Testing with 10 clusters
Testing with 15 clusters
Testing with 20 clusters
Testing with 25 clusters
Testing with 30 clusters


In [51]:
results_avg = dict()
for k in results.keys():
    accuracies, rand_indices = zip(*results[k])
    results_avg[k] = (np.mean(accuracies), np.mean(rand_indices))
results_avg

{5: (0.42612612612612616, 0.68445743040337637),
 10: (0.463963963963964, 0.7381624868111355),
 15: (0.49279279279279276, 0.7727944160376593),
 20: (0.51891891891891895, 0.78395422449476515),
 25: (0.51081081081081092, 0.78098368638909177),
 30: (0.5, 0.77091145199253297)}